In [56]:
GPU_device_id = str(3)
model_id_save_as = 'daepretrain-full-final'
architecture_id = 'final-models/learningcurve-dnn-full-final-features'
model_class_id = 'DAE'
training_dataset_id = '../dataset_generation/hyperparametersearch_dataset_200keV_full_log10time_1000.npy'
difficulty_setting = 'full'

earlystop_patience = 50
num_epochs = 2000

In [16]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)



#### Import model, training function 

In [17]:
from annsa.model_classes import build_dae_model, compile_model, mean_normalized_kl_divergence
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

## Training Data Construction

In [18]:
training_dataset = np.load(training_dataset_id, allow_pickle=True)
training_source_spectra, training_background_spectra, training_keys = dataset_to_spectrakeys(training_dataset,
                                                                                             sampled=False,
                                                                                             separate_background=True)

## Load Model

In [19]:
model_features = load_features(architecture_id)
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.learning_rate = model_features.learining_rate
model_features.input_dim = 1024

In [20]:
dae_features = model_features.to_dae_model_features()
dae_features.metrics = ['mse']

## Train network

# Scale input data

In [21]:
training_input = np.random.poisson(training_source_spectra+training_background_spectra)
training_output = training_source_spectra

training_input = dae_features.scaler.transform(training_input)
training_output = dae_features.scaler.transform(training_output)


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:161: RuntimeWarning: invalid value encountered in sqrt
  **(kw_args if kw_args else {}))


In [22]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=earlystop_patience,
    mode='min',
    restore_best_weights=True)

csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'.log')

In [23]:
mlb=LabelBinarizer()
model = compile_model(
            build_dae_model,
            dae_features)

output = model.fit(
    x=training_input,
    y=training_output,
    batch_size=model_features.batch_size,
    validation_split=0.1,
    epochs=500,
    verbose=1,
    shuffle=True,
    callbacks=[earlystop_callback, ],
)


Train on 27000 samples, validate on 3000 samples
Epoch 1/500
27000/27000 [==============================] - 1s 27us/sample - loss: 140.0044 - mean_squared_error: 140.0044 - val_loss: 41.5876 - val_mean_squared_error: 41.5876
Epoch 2/500
27000/27000 [==============================] - 0s 17us/sample - loss: 50.9727 - mean_squared_error: 50.9728 - val_loss: 28.9551 - val_mean_squared_error: 28.9551
Epoch 3/500
27000/27000 [==============================] - 0s 17us/sample - loss: 33.7771 - mean_squared_error: 33.7771 - val_loss: 21.9606 - val_mean_squared_error: 21.9606
Epoch 4/500
27000/27000 [==============================] - 0s 17us/sample - loss: 25.1494 - mean_squared_error: 25.1494 - val_loss: 20.1289 - val_mean_squared_error: 20.1289
Epoch 5/500
27000/27000 [==============================] - 0s 17us/sample - loss: 20.3494 - mean_squared_error: 20.3494 - val_loss: 18.3879 - val_mean_squared_error: 18.3879
Epoch 6/500
27000/27000 [==============================] - 0s 17us/sample - los

27000/27000 [==============================] - 0s 17us/sample - loss: 6.0648 - mean_squared_error: 6.0648 - val_loss: 12.3285 - val_mean_squared_error: 12.3285
Epoch 20/500
27000/27000 [==============================] - 0s 17us/sample - loss: 5.8705 - mean_squared_error: 5.8705 - val_loss: 12.6493 - val_mean_squared_error: 12.6493
Epoch 21/500
27000/27000 [==============================] - 0s 17us/sample - loss: 5.7008 - mean_squared_error: 5.7008 - val_loss: 12.2729 - val_mean_squared_error: 12.2729
Epoch 22/500
27000/27000 [==============================] - 0s 17us/sample - loss: 5.5014 - mean_squared_error: 5.5014 - val_loss: 11.9705 - val_mean_squared_error: 11.9705
Epoch 23/500
27000/27000 [==============================] - 0s 17us/sample - loss: 5.3387 - mean_squared_error: 5.3387 - val_loss: 11.8219 - val_mean_squared_error: 11.8219
Epoch 24/500
27000/27000 [==============================] - 0s 17us/sample - loss: 5.1915 - mean_squared_error: 5.1915 - val_loss: 11.4737 - val_mea

27000/27000 [==============================] - 0s 17us/sample - loss: 3.7808 - mean_squared_error: 3.7808 - val_loss: 9.9967 - val_mean_squared_error: 9.9967
Epoch 39/500
27000/27000 [==============================] - 0s 17us/sample - loss: 3.7222 - mean_squared_error: 3.7222 - val_loss: 9.6910 - val_mean_squared_error: 9.6910
Epoch 40/500
27000/27000 [==============================] - 0s 17us/sample - loss: 3.6331 - mean_squared_error: 3.6331 - val_loss: 9.4916 - val_mean_squared_error: 9.4916
Epoch 41/500
27000/27000 [==============================] - 0s 17us/sample - loss: 3.5796 - mean_squared_error: 3.5796 - val_loss: 9.5043 - val_mean_squared_error: 9.5043
Epoch 42/500
27000/27000 [==============================] - 0s 17us/sample - loss: 3.5173 - mean_squared_error: 3.5173 - val_loss: 9.4863 - val_mean_squared_error: 9.4863
Epoch 43/500
27000/27000 [==============================] - 0s 17us/sample - loss: 3.4533 - mean_squared_error: 3.4533 - val_loss: 9.2803 - val_mean_squared_e

27000/27000 [==============================] - 0s 17us/sample - loss: 2.8266 - mean_squared_error: 2.8266 - val_loss: 7.9607 - val_mean_squared_error: 7.9607
Epoch 58/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.7916 - mean_squared_error: 2.7916 - val_loss: 8.0021 - val_mean_squared_error: 8.0021
Epoch 59/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.7575 - mean_squared_error: 2.7575 - val_loss: 7.7708 - val_mean_squared_error: 7.7708
Epoch 60/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.7313 - mean_squared_error: 2.7313 - val_loss: 7.7376 - val_mean_squared_error: 7.7376
Epoch 61/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.7076 - mean_squared_error: 2.7076 - val_loss: 7.7942 - val_mean_squared_error: 7.7942
Epoch 62/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.7120 - mean_squared_error: 2.7120 - val_loss: 7.4725 - val_mean_squared_e

27000/27000 [==============================] - 0s 17us/sample - loss: 2.3750 - mean_squared_error: 2.3750 - val_loss: 6.7924 - val_mean_squared_error: 6.7924
Epoch 77/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.3188 - mean_squared_error: 2.3188 - val_loss: 6.9921 - val_mean_squared_error: 6.9921
Epoch 78/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.2972 - mean_squared_error: 2.2972 - val_loss: 6.9610 - val_mean_squared_error: 6.9610
Epoch 79/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.3229 - mean_squared_error: 2.3229 - val_loss: 6.8290 - val_mean_squared_error: 6.8290
Epoch 80/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.2334 - mean_squared_error: 2.2334 - val_loss: 6.9527 - val_mean_squared_error: 6.9527
Epoch 81/500
27000/27000 [==============================] - 0s 17us/sample - loss: 2.2284 - mean_squared_error: 2.2284 - val_loss: 6.9377 - val_mean_squared_e

...

27000/27000 [==============================] - 0s 17us/sample - loss: 0.9729 - mean_squared_error: 0.9729 - val_loss: 9.3753 - val_mean_squared_error: 9.3753
Epoch 343/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.9091 - mean_squared_error: 0.9091 - val_loss: 8.5188 - val_mean_squared_error: 8.5188
Epoch 344/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.9005 - mean_squared_error: 0.9005 - val_loss: 9.0488 - val_mean_squared_error: 9.0488
Epoch 345/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8996 - mean_squared_error: 0.8996 - val_loss: 8.4027 - val_mean_squared_error: 8.4027
Epoch 346/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.9012 - mean_squared_error: 0.9012 - val_loss: 9.0028 - val_mean_squared_error: 9.0028
Epoch 347/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8984 - mean_squared_error: 0.8984 - val_loss: 9.1148 - val_mean_squa

27000/27000 [==============================] - 0s 17us/sample - loss: 0.8718 - mean_squared_error: 0.8718 - val_loss: 9.2571 - val_mean_squared_error: 9.2571
Epoch 362/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8880 - mean_squared_error: 0.8880 - val_loss: 9.0945 - val_mean_squared_error: 9.0945
Epoch 363/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8860 - mean_squared_error: 0.8860 - val_loss: 10.4085 - val_mean_squared_error: 10.4085
Epoch 364/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8842 - mean_squared_error: 0.8842 - val_loss: 9.7360 - val_mean_squared_error: 9.7360
Epoch 365/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8753 - mean_squared_error: 0.8753 - val_loss: 9.3513 - val_mean_squared_error: 9.3513
Epoch 366/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8769 - mean_squared_error: 0.8769 - val_loss: 9.5535 - val_mean_sq

27000/27000 [==============================] - 0s 17us/sample - loss: 0.8699 - mean_squared_error: 0.8699 - val_loss: 9.4114 - val_mean_squared_error: 9.4114
Epoch 381/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8529 - mean_squared_error: 0.8529 - val_loss: 9.9762 - val_mean_squared_error: 9.9762
Epoch 382/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8736 - mean_squared_error: 0.8736 - val_loss: 9.9351 - val_mean_squared_error: 9.9351
Epoch 383/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8368 - mean_squared_error: 0.8368 - val_loss: 9.8983 - val_mean_squared_error: 9.8983
Epoch 384/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8500 - mean_squared_error: 0.8500 - val_loss: 10.0227 - val_mean_squared_error: 10.0227
Epoch 385/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8628 - mean_squared_error: 0.8628 - val_loss: 11.0705 - val_mean_s

27000/27000 [==============================] - 0s 17us/sample - loss: 0.8110 - mean_squared_error: 0.8110 - val_loss: 10.8007 - val_mean_squared_error: 10.8007
Epoch 418/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8047 - mean_squared_error: 0.8047 - val_loss: 11.5050 - val_mean_squared_error: 11.5050
Epoch 419/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8356 - mean_squared_error: 0.8356 - val_loss: 10.8324 - val_mean_squared_error: 10.8324
Epoch 420/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8213 - mean_squared_error: 0.8213 - val_loss: 11.0789 - val_mean_squared_error: 11.0789
Epoch 421/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.7933 - mean_squared_error: 0.7933 - val_loss: 10.5517 - val_mean_squared_error: 10.5517
Epoch 422/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8122 - mean_squared_error: 0.8122 - val_loss: 11.2096 - va

KeyboardInterrupt: 

In [ ]:
model.save('./final-models-keras/'+model_id_save_as+'.hdf5')